# Perform regression

## Make FASTA files with mutations on Wuhan WT

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Seq import MutableSeq

In [2]:
with open('wuhan_spike.fasta','r') as in_fasta:
    for record in SeqIO.parse(in_fasta,'fasta'):
        wuhan = record.seq

In [3]:
def make_mutation(mutation):
    WT = mutation[0]
    mutated = mutation[-1]
    site = int(mutation[1:-1])
    seq = MutableSeq(str(wuhan))
    if seq[site-1] == WT:
        seq[site-1] = mutated
        seq = seq[318:541]
        return SeqRecord(seq, id='Wuhan_RBD_'+mutation,description="")
    else:
        "Mutation does not match"

In [4]:
with open('mutated_seqs.fasta','w') as out:
    with open('mutations.txt','r') as i:
        lines = i.readlines()
        lines = [line.rstrip("\n") for line in lines]
        
        for l in lines:
            x = make_mutation(l)
            SeqIO.write([x], out, "fasta")

## Prepare data and fit to logistic regression

In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

In [2]:
df = pd.read_csv('sheet.csv')
df['outcome'] = df['outcome'].astype(int)

In [3]:
with open('mutated_esms/list.txt','r') as f:
    l = f.readlines()
    lines = [line.rstrip("\n") for line in l]
lines = lines[:-1]

In [4]:
X = []
Y = []
for l in lines:
    x1 = torch.load('mutated_esms/Wuhan_RBD_'+l+'.pt')['mean_representations'][33]
    x2 = df[df['Mutations'] == l][['RBD','ACE2','class','size','polarity','charge','outcome']]
    x2 = torch.tensor(x2.values).flatten()
    x = torch.concatenate((x1,x2.flatten()))
    y = x[-1]
    x = x[:-1]
    X.append(x)
    Y.append(y)

In [5]:
Y = np.array(Y)

In [6]:
x_arr = []
for i in X:
    x = np.array(i)
    x_arr.append(x)
X = np.array(x_arr)

In [7]:
X = np.nan_to_num(X, nan=0)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [8]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_prob = logreg.predict_proba(X_test)

/home/akshay/apps/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
log_loss(Y_test, Y_prob)

0.18822475616603548

In [10]:
roc_auc_score(Y_test, Y_prob[:, 1])

0.9523809523809523

In [11]:
accuracy_score(Y_test,np.argmax(Y_prob,-1))

0.9545454545454546

In [12]:
recall_score(Y_test, np.argmax(Y_prob,-1))

0.0

In [13]:
f1_score(Y_test, np.argmax(Y_prob,-1))

0.0

In [16]:
import joblib
joblib.dump(logreg,'logistic_regression.pkl')

['logistic_regression.pkl']

In [20]:
X_test.shape

(22, 1286)

## Use a MLP

In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df = pd.read_csv('sheet.csv')
df['outcome'] = df['outcome'].astype(int)

In [3]:
with open('mutated_esms/list.txt','r') as f:
    l = f.readlines()
    lines = [line.rstrip("\n") for line in l]
lines = lines[:-1]

In [4]:
X = []
Y = []
for l in lines:
    x1 = torch.load('mutated_esms/Wuhan_RBD_'+l+'.pt')['mean_representations'][33]
    x2 = df[df['Mutations'] == l][['RBD','ACE2','class','size','polarity','charge','outcome']]
    x2 = torch.tensor(x2.values).flatten()
    x = torch.concatenate((x1,x2.flatten()))
    y = x[-1]
    x = x[:-1]
    X.append(x)
    Y.append(y)

In [5]:
Y = np.array(Y)

In [6]:
x_arr = []
for i in X:
    x = np.array(i)
    x_arr.append(x)
X = np.array(x_arr)

In [7]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.Sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.Sigmoid(out)
        return out

In [9]:
X = np.nan_to_num(X, nan=0)
# Assuming X is your list of 213 vectors and Y is your list of labels (zeros and ones)
# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, Y_tensor, test_size=0.1)

# Initialize the model, loss function, and optimizer
input_size = X.shape[1]  # Assuming X has shape (number of samples, number of features)
hidden_size = 32
output_size = 1
model = MLP(input_size, hidden_size, output_size)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Create DataLoader for training set
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))  # BCELoss expects labels to have shape (batch_size, 1)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    with torch.no_grad():
        model.eval()
        outputs = model(X_test)
        print(log_loss(Y_test, outputs))
        print(roc_auc_score(Y_test, outputs))
        #predicted = (outputs >= 0.5).float()  # Convert probabilities to binary predictions
        #accuracy = (predicted == Y_test.unsqueeze(1)).float().mean().item()
        #print(f'Accuracy: {accuracy:.4f}')

# Evaluate the model
with torch.no_grad():
    model.eval()
    outputs = model(X_test)
    predicted = (outputs >= 0.5).float()  # Convert probabilities to binary predictions
    accuracy = (predicted == Y_test.unsqueeze(1)).float().mean().item()
    print(f'Accuracy: {accuracy:.4f}')

Epoch [1/10], Loss: 0.5771
0.5623595077155839
0.19444444444444445
Epoch [2/10], Loss: 0.3764
0.4939753380625028
0.16666666666666666
Epoch [3/10], Loss: 0.3944
0.48189175485580477
0.15277777777777776
Epoch [4/10], Loss: 0.5766
0.49763921712725523
0.15277777777777776
Epoch [5/10], Loss: 0.4565
0.5069449802243581
0.16666666666666666
Epoch [6/10], Loss: 0.3891
0.5025961200575796
0.19444444444444442
Epoch [7/10], Loss: 0.3241
0.4931491515531023
0.2361111111111111
Epoch [8/10], Loss: 0.3226
0.4871351863012468
0.23611111111111113
Epoch [9/10], Loss: 0.3844
0.4852582883353747
0.23611111111111113
Epoch [10/10], Loss: 0.5440
0.4857687129740903
0.2638888888888889
Accuracy: 0.8182


## Use XGBoost

In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

In [2]:
df = pd.read_csv('sheet.csv')
df['outcome'] = df['outcome'].astype(int)

In [3]:
with open('mutated_esms/list.txt','r') as f:
    l = f.readlines()
    lines = [line.rstrip("\n") for line in l]
lines = lines[:-1]

In [4]:
X = []
Y = []
for l in lines:
    x1 = torch.load('mutated_esms/Wuhan_RBD_'+l+'.pt')['mean_representations'][33]
    x2 = df[df['Mutations'] == l][['RBD','ACE2','class','size','polarity','charge','outcome']]
    x2 = torch.tensor(x2.values).flatten()
    x = torch.concatenate((x1,x2.flatten()))
    y = x[-1]
    x = x[:-1]
    X.append(x)
    Y.append(y)

In [5]:
Y = np.array(Y)

In [6]:
x_arr = []
for i in X:
    x = np.array(i)
    x_arr.append(x)
X = np.array(x_arr)

In [23]:
X = np.nan_to_num(X, nan=0)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [24]:
model = xgb.XGBClassifier()
model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [25]:
Y_pred = model.predict(X_test)

In [26]:
accuracy_score(Y_test, Y_pred)

0.7727272727272727

In [27]:
roc_auc_score(Y_test, Y_pred)

0.5

In [28]:
f1_score(Y_test, Y_pred)

0.0

In [29]:
recall_score(Y_test, Y_pred)

0.0

In [22]:
precision_score(Y_test, Y_pred)

/home/akshay/apps/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [31]:
import joblib

In [32]:
joblib.dump(model,'xgboost.pkl')

['xgboost.pkl']

## Decision tree

In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

In [2]:
df = pd.read_csv('sheet.csv')
df['outcome'] = df['outcome'].astype(int)

In [3]:
with open('mutated_esms/list.txt','r') as f:
    l = f.readlines()
    lines = [line.rstrip("\n") for line in l]
lines = lines[:-1]

In [4]:
X = []
Y = []
for l in lines:
    x1 = torch.load('mutated_esms/Wuhan_RBD_'+l+'.pt')['mean_representations'][33]
    x2 = df[df['Mutations'] == l][['RBD','ACE2','class','size','polarity','charge','outcome']]
    x2 = torch.tensor(x2.values).flatten()
    x = torch.concatenate((x1,x2.flatten()))
    y = x[-1]
    x = x[:-1]
    X.append(x)
    Y.append(y)

In [5]:
Y = np.array(Y)

In [6]:
x_arr = []
for i in X:
    x = np.array(i)
    x_arr.append(x)
X = np.array(x_arr)

In [13]:
X = np.nan_to_num(X, nan=0)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [14]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

In [15]:
accuracy_score(Y_test,Y_pred)

0.8181818181818182

In [16]:
precision_score(Y_test,Y_pred)

1.0

In [17]:
recall_score(Y_test, Y_pred)

0.3333333333333333

In [18]:
f1_score(Y_test, Y_pred)

0.5

In [19]:
import joblib

In [20]:
joblib.dump(clf,'decisiontree.pkl')

['decisiontree.pkl']

# Tests

In [15]:
roc_auc_score(Y_test, Y_prob[:, 1])

0.7307692307692308

In [69]:
import pandas as pd

In [91]:
df = pd.read_csv('sheet.csv')
df['outcome'] = df['outcome'].astype(int)

In [135]:
df

,Mutations,prev,where,RBD,ACE2,class,size,polarity,charge,mutations,outcome
0,G339H,YES,"B.1.1.529/XBB,BA.2.86",0.118639,0.0000,5,1,1,0,GH,1
1,Q493R,YES,"B.1.1.529/BA.1,BA.2",-0.017792,-0.4068,2,0,0,1,QR,1
2,S371L,YES,B.1.1.529/BA.1,-0.258961,0.0000,4,0,-1,0,SL,1
3,G446S,YES,"B.1.1.529/BA.1,BA.2,BA.2.75,BA.2.86,CM.8,XBB",3.274680,-0.4243,3,1,1,0,GS,1
4,R346T,YES,"B.1.1.529/BS.1,FV.1,BM.1,BM.4.1.1",1.165800,0.0000,3,-1,0,-1,RT,1
...,...,...,...,...,...,...,...,...,...,...,...
208,V445L,NaN,NaN,-0.860684,-0.1809,3,0,0,0,VL,0
209,P507A,NaN,NaN,2.369770,-0.0026,5,0,0,0,PA,0
210,G446D,NaN,NaN,3.011890,-0.4027,3,1,1,-1,GD,0
211,F464L,NaN,NaN,1.867110,0.0000,5,-1,0,0,FL,0


In [92]:
with open('mutated_esms/list.txt','r') as f:
    l = f.readlines()
    lines = [line.rstrip("\n") for line in l]

In [94]:
df[df['Mutations'] == lines[1]]

,Mutations,prev,where,RBD,ACE2,class,size,polarity,charge,mutations,outcome
174,A344S,NaN,NaN,-0.326079,0.0,5,0,1,0,AS,0


In [95]:
x = torch.load('mutated_esms/Wuhan_RBD_'+lines[0]+'.pt')['mean_representations'][33]

In [96]:
y = df[df['Mutations'] == lines[0]][['RBD','ACE2','class','size','polarity','charge','outcome']]

In [103]:
y = torch.tensor(y.values)

RuntimeError: Could not infer dtype of builtin_function_or_method

In [101]:
x.shape

torch.Size([1280])

In [102]:
y.shape

torch.Size([1, 7])

In [107]:
torch.concatenate((x,y.flatten()))[:-1].shape

torch.Size([1286])

In [108]:
X = []
Y = []
for l in lines:
    x1 = torch.load('mutated_esms/Wuhan_RBD_'+lines[0]+'.pt')['mean_representations'][33]
    x2 = df[df['Mutations'] == lines[0]][['RBD','ACE2','class','size','polarity','charge','outcome']]
    x2 = torch.tensor(x2.values).flatten()
    x = torch.concatenate((x1,x2.flatten()))
    y = x[-1]
    x = x[:-1]
    X.append(x)
    Y.append(y)

In [109]:
len(X)

213

In [110]:
len(Y)

213

In [111]:
X[0].shape

torch.Size([1286])

In [149]:
print(np.where(np.isnan(X)))

(array([39]), array([1281]))


In [ ]:
Y = np.array(Y)

In [132]:
x_arr = []
for i in X:
    x = np.array(i)
    x_arr.append(x)
X = np.array(x_arr)    

In [150]:
X = np.nan_to_num(X, nan=0)

In [151]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [152]:
logreg = LogisticRegression()

# Train the model
logreg.fit(X_train, Y_train)

# Predict on the test set
Y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = (Y_pred == Y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.9069767441860465


In [21]:
import torch

In [22]:
x = torch.load('mutated_esms/Wuhan_RBD_A344D.pt')

In [26]:
x['mean_representations'][33].shape

torch.Size([1280])

In [10]:
with open('mutated_seqs.fasta','r') as in_fasta:
    seqs = []
    for record in SeqIO.parse(in_fasta,'fasta'):
        seqs.append(MutableSeq(str(record.seq)))

In [16]:
with open('mutations.txt','r') as i:
    lines = i.readlines()
    lines = [line.rstrip("\n") for line in lines]

In [20]:
for i,l in enumerate(lines):
    mutated = l[-1]
    site = int(l[1:-1])
    if seqs[i][site-319] != mutated:
        print('Not OK')

In [8]:
record.name

'Wuhan_RBD_F486A'

In [63]:
with open('mutations.txt','r') as i:
    lines = i.readlines()
    lines = [line.rstrip("\n") for line in lines]
    

In [51]:
x = 'G504F'

In [55]:
int(x[1:-1])

504

In [1]:
!python --version

Python 3.11.5


In [2]:
!esm-extract

usage: esm-extract [-h] [--toks_per_batch TOKS_PER_BATCH]
                   [--repr_layers REPR_LAYERS [REPR_LAYERS ...]] --include
                   {mean,per_tok,bos,contacts}
                   [{mean,per_tok,bos,contacts} ...]
                   [--truncation_seq_length TRUNCATION_SEQ_LENGTH] [--nogpu]
                   model_location fasta_file output_dir
esm-extract: error: the following arguments are required: model_location, fasta_file, output_dir, --include


In [3]:
from Bio import SeqIO

In [82]:
def modify_sequence(sequence,site,initial_mutation,final_mutation):
    modified_sequence = sequence.replace()

In [83]:
input_fasta = 'wuhan_spike.fasta'

In [84]:
with open(input_fasta,'r') as input_handle:
    for record in SeqIO.parse(input_fasta,'fasta'):
        in_fasta = record

In [85]:
record.seq

Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT')

In [86]:
from Bio.Seq import Seq

In [87]:
from Bio.SeqRecord import SeqRecord

In [88]:
x = record.seq

In [89]:
from Bio.Seq import MutableSeq

In [90]:
y = MutableSeq(str(x))

In [30]:
y[450-1] = 'D'

In [104]:
y[355]

'K'

In [93]:
y[450-1] == 'N'

True

In [49]:
z = y[318:541]

In [50]:
z

MutableSeq('RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSA...VNF')

In [36]:
SeqRecord(y)

SeqRecord(seq=MutableSeq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...HYT'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])

In [34]:
record.id

'Wuhan_spike'